<a href="https://colab.research.google.com/github/wanwanliang/ML_Projects/blob/main/DL_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

## Load datasets

In [5]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full/255.0
X_test = X_test/255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
def split_dataset(X,y):
  y_5_or_6 = (y==5)|(y==6)
  y_A = y[~y_5_or_6]
  y_A[y_A>6] -= 2
  y_B = (y[y_5_or_6]==6).astype(np.float32)

  return((X[~y_5_or_6], y_A), (X[y_5_or_6], y_B))

In [8]:
(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [9]:
X_train_A.shape

(43986, 28, 28)

In [10]:
X_train_B.shape

(200, 28, 28)

In [59]:
tf.random.set_seed(42)
np.random.seed(42)

In [60]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))

for n_hidden in (300, 100, 50, 50, 50):
  model_A.add(keras.layers.Dense(n_hidden, activation='selu'))

model_A.add(keras.layers.Dense(8, activation='softmax'))

In [61]:
model_A.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.SGD(lr=1e-3), metrics=[keras.metrics.categorical_accuracy])

In [62]:
A_history = model_A.fit(X_train_A, y_train_A, epochs=20, validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.5926 - categorical_accuracy: 0.1262 - val_loss: 0.3890 - val_categorical_accuracy: 0.1131
Epoch 2/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3523 - categorical_accuracy: 0.1226 - val_loss: 0.3289 - val_categorical_accuracy: 0.1136
Epoch 3/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.3170 - categorical_accuracy: 0.1235 - val_loss: 0.3013 - val_categorical_accuracy: 0.1106
Epoch 4/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2972 - categorical_accuracy: 0.1242 - val_loss: 0.2892 - val_categorical_accuracy: 0.1163
Epoch 5/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2834 - categorical_accuracy: 0.1242 - val_loss: 0.2774 - val_categorical_accuracy: 0.1156
Epoch 6/20
1375/1375 [==============================] - 4s 3ms/step - loss: 0.2729 - categorical_accuracy: 0.1243 - val_loss: 0.2733 - val_categorical_accuracy:

In [63]:
model_A.save('model_A.h5')

In [64]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300, 100, 50, 50, 50):
  model_B.add(keras.layers.Dense(n_hidden, activation='selu'))
model_B.add(keras.layers.Dense(1, activation='sigmoid'))

In [65]:
model_B.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=1e-3), metrics=['accuracy'])

In [66]:
B_history = model_B.fit(X_train_B, y_train_B, epochs=20, validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 0s 33ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 13ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 13ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 13ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 13ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 12ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 12ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

In [67]:
model_A.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_15 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_16 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_17 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_18 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_19 (Dense)             (None, 8)                

In [68]:
model_B.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_21 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_22 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_23 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_24 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_25 (Dense)             (None, 1)                

In [69]:
model_A = keras.models.load_model('model_A.h5')
model_A_clone = keras.models.clone_model(model_A) 
model_A_clone.set_weights(model_A.get_weights())

In [70]:
print(len(model_A.layers))
model_A.layers

7


In [71]:
transfer_model = keras.models.Sequential(model_A.layers[:-1])
transfer_model.add(keras.layers.Dense(1, activation='sigmoid'))
transfer_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_15 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_16 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_17 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_18 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_26 (Dense)             (None, 1)                

In [72]:
for layer in transfer_model.layers[:-1]:
  layer.trainable = False

transfer_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(lr=1e-3), metrics=['accuracy'])

In [73]:
tm_history = transfer_model.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 28ms/step - loss: 0.4481 - accuracy: 0.8350 - val_loss: 0.4436 - val_accuracy: 0.8174
Epoch 2/4
7/7 [==============================] - 0s 10ms/step - loss: 0.4157 - accuracy: 0.8500 - val_loss: 0.4145 - val_accuracy: 0.8306
Epoch 3/4
7/7 [==============================] - 0s 11ms/step - loss: 0.3870 - accuracy: 0.8600 - val_loss: 0.3880 - val_accuracy: 0.8458
Epoch 4/4
7/7 [==============================] - 0s 11ms/step - loss: 0.3610 - accuracy: 0.8700 - val_loss: 0.3650 - val_accuracy: 0.8529


In [74]:
for layer in transfer_model.layers[:-1]:
  layer.trainable=True
transfer_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(lr=1e-3), metrics=['accuracy'])

In [75]:
tm_history = transfer_model.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))

Epoch 1/16
7/7 [==============================] - 0s 30ms/step - loss: 0.2943 - accuracy: 0.9250 - val_loss: 0.2487 - val_accuracy: 0.9523
Epoch 2/16
7/7 [==============================] - 0s 11ms/step - loss: 0.2061 - accuracy: 0.9850 - val_loss: 0.1968 - val_accuracy: 0.9797
Epoch 3/16
7/7 [==============================] - 0s 10ms/step - loss: 0.1635 - accuracy: 0.9900 - val_loss: 0.1655 - val_accuracy: 0.9899
Epoch 4/16
7/7 [==============================] - 0s 11ms/step - loss: 0.1360 - accuracy: 1.0000 - val_loss: 0.1452 - val_accuracy: 0.9919
Epoch 5/16
7/7 [==============================] - 0s 11ms/step - loss: 0.1183 - accuracy: 1.0000 - val_loss: 0.1302 - val_accuracy: 0.9929
Epoch 6/16
7/7 [==============================] - 0s 13ms/step - loss: 0.1046 - accuracy: 1.0000 - val_loss: 0.1188 - val_accuracy: 0.9919
Epoch 7/16
7/7 [==============================] - 0s 11ms/step - loss: 0.0944 - accuracy: 1.0000 - val_loss: 0.1098 - val_accuracy: 0.9919
Epoch 8/16
7/7 [===========

In [76]:
transfer_model.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0635 - accuracy: 0.9980


[0.06347920745611191, 0.9980000257492065]

In [77]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.1408 - accuracy: 0.9705


[0.14084088802337646, 0.9704999923706055]